In [ ]:
!pip install -q ultralytics torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 24.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os

# Path to the zip file in Google Drive
zip_path = "/content/drive/My Drive/Data/Augmented_trash.zip"

# Directory to extract to
extract_dir = "/content"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Extracted contents to {extract_dir}")

print(os.listdir("/content"))


Extracted contents to /content
['.config', 'drive', 'Augmented_trash', 'sample_data']


In [ ]:
import shutil

# Current dataset path
source_path = "/content/Augmented_trash"

# Target dataset path
target_path = "/content/datasets/Augmented_trash"

# Move the dataset
shutil.move(source_path, target_path)


'/content/datasets/Augmented_trash'

In [ ]:
from ultralytics import YOLO
import torch


def train_yolo11_detector(
    data_yaml="dataset.yaml",
    model_size="n",
    epochs=100,
    imgsz=640,
    batch=16,
    workers=2,
    project="trash_detection",
    name="train_yolo11_colab",
    device=None
):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Training on device: {device}")

    # Load the YOLO model
    model = YOLO(f"yolo11{model_size}.pt")  # Load pretrained weights

    # Train the model
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        workers=workers,
        project=project,
        name=name,
        exist_ok=True,
        pretrained=True,
        amp=True,
        device=device,
        save_period=3,  # Save every 3 epochs
        mosaic=1.0,
        mixup=0.0,
        fliplr=0.5,
        translate=0.1,
        scale=0.5,
        degrees=10.0,
    )

    # Save weights to Google Drive
    source_path = f"{project}/{name}/weights"
    target_path = "/content/drive/My Drive/trash_detection_checkpoints"
    os.makedirs(target_path, exist_ok=True)

    for file in os.listdir(source_path):
        shutil.copy(os.path.join(source_path, file), target_path)

    print(f"Checkpoints saved to Google Drive at {target_path}")
    return results


if __name__ == "__main__":
    data_yaml_path = "/content/datasets/Augmented_trash/dataset.yaml"
    if not torch.cuda.is_available():
        print("Warning: GPU not available! Training will proceed on CPU.")
    else:
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")

    results = train_yolo11_detector(
        data_yaml=data_yaml_path,
        model_size="n",
        epochs=10,
        imgsz=640,
        batch=16,
        project="trash_detection",
        name="yolo11_training",
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    print("\nTraining Complete!")
    print(f"Final mAP50-95: {results.results_dict.get('metrics/mAP50-95', 0):.3f}")
    print(f"Final mAP50: {results.results_dict.get('metrics/mAP50', 0):.3f}")


Using GPU: Tesla T4
Training on device: cuda
Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/datasets/Augmented_trash/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=3, cache=False, device=cuda, workers=2, project=trash_detection, name=yolo11_training, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

train: Scanning /content/datasets/Augmented_trash/labels/train.cache... 43016 images, 0 backgrounds, 0 corrupt: 100%|██████████| 43016/43016 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/Augmented_trash/labels/val.cache... 10729 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10729/10729 [00:00<?, ?it/s]


Plotting labels to trash_detection/yolo11_training/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to trash_detection/yolo11_training
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      2.57G     0.3059     0.1973     0.8182          8        640: 100%|██████████| 2689/2689 [14:20<00:00,  3.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 336/336 [01:06<00:00,  5.03it/s]

                   all      10729      10729      0.999      0.999      0.995      0.968



10 epochs completed in 2.642 hours.
Optimizer stripped from trash_detection/yolo11_training/weights/last.pt, 5.5MB
Optimizer stripped from trash_detection/yolo11_training/weights/best.pt, 5.5MB

Validating trash_detection/yolo11_training/weights/best.pt...
Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 336/336 [01:09<00:00,  4.82it/s]


                   all      10729      10729      0.999      0.999      0.995      0.968
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to trash_detection/yolo11_training
Checkpoints saved to Google Drive at /content/drive/My Drive/trash_detection_checkpoints

Training Complete!
Final mAP50-95: 0.000
Final mAP50: 0.000


In [ ]:
import shutil
from google.colab import files

folder_path = "/content/trash_detection"

zip_path = "/content/trash_detection.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>